## **TODO COLAB**:
- create folder `isplutils` and import `split.py`, `utils.py` and `data.py` inside
- create folder `architectures` and import `fornet.py` inside

In [4]:
import argparse
import os
import shutil
import warnings

import albumentations as A
import numpy as np
import pandas as pd
import torch
import torch.multiprocessing
from torchvision.transforms import ToPILImage, ToTensor

from isplutils import utils, split

torch.multiprocessing.set_sharing_strategy('file_system')
import torch.nn as nn
#!pip install --upgrade --force-reinstall --no-deps albumentations==0.4.6
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import roc_auc_score
#!pip install tensorboardX
from tensorboardX import SummaryWriter
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import ImageChops, Image
from typing import List, Dict, Tuple

#!pip install --upgrade efficientnet-pytorch
from architectures import fornet

from isplutils.data import FrameFaceIterableDataset, load_face

In [5]:
GPU = False
device = 'cpu'
if GPU:
    !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
    !pip install gputil
    !pip install psutil
    !pip install humanize
    import psutil, humanize, os, GPUtil
    gpu_detail = GPUtil.getGPUs()[0]
    print(GPUtil.getGPUs())
    process = psutil.Process(os.getpid())
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".\
          format(gpu_detail.memoryFree, gpu_detail.memoryUsed, gpu_detail.memoryUtil*100, gpu_detail.memoryTotal))
    # Make sure to enable GPU acceleration!
    device = 'cuda'

In [6]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [22]:
def tb_attention(tb: SummaryWriter,
                 tag: str,
                 iteration: int,
                 net: nn.Module,
                 device: torch.device,
                 patch_size_load: int,
                 face_crop_scale: str,
                 val_transformer: A.BasicTransform,
                 root: str,
                 record: pd.Series):
    # Crop face
    sample_t = load_face(record=record, root=root, size=patch_size_load, scale=face_crop_scale,
                         transformer=val_transformer)
    sample_t_clean = load_face(record=record, root=root, size=patch_size_load, scale=face_crop_scale,
                               transformer=ToTensorV2())
    if torch.cuda.is_available():
        sample_t = sample_t.cuda(device)
    # Transform
    # Feed to net
    with torch.no_grad():
        att: torch.Tensor = net.get_attention(sample_t.unsqueeze(0))[0].cpu()
    att_img: Image.Image = ToPILImage()(att)
    sample_img = ToPILImage()(sample_t_clean)
    att_img = att_img.resize(sample_img.size, resample=Image.NEAREST).convert('RGB')
    sample_att_img = ImageChops.multiply(sample_img, att_img)
    sample_att = ToTensor()(sample_att_img)
    tb.add_image(tag=tag, img_tensor=sample_att, global_step=iteration)


def batch_forward(net: nn.Module, device: torch.device, criterion, data: torch.Tensor, labels: torch.Tensor) -> (
        torch.Tensor, float, int):
    data = data.to(device)
    labels = labels.to(device)
    out = net(data)
    pred = torch.sigmoid(out).detach().cpu().numpy()
    loss = criterion(out, labels)
    return loss, pred


def validation_routine(net, device, val_loader, criterion, tb, iteration, tag: str, loader_len_norm: int = None):
    net.eval()
    loader_len_norm = loader_len_norm if loader_len_norm is not None else val_loader.batch_size
    val_num = 0
    val_loss = 0.
    pred_list = list()
    labels_list = list()
    for val_data in tqdm(val_loader, desc='Validation', leave=False, total=len(val_loader) // loader_len_norm):
        batch_data, batch_labels = val_data

        val_batch_num = len(batch_labels)
        labels_list.append(batch_labels.flatten())
        with torch.no_grad():
            val_batch_loss, val_batch_pred = batch_forward(net, device, criterion, batch_data,
                                                           batch_labels)
        pred_list.append(val_batch_pred.flatten())
        val_num += val_batch_num
        val_loss += val_batch_loss.item() * val_batch_num

    # Logging
    val_loss /= val_num
    tb.add_scalar('{}/loss'.format(tag), val_loss, iteration)

    if isinstance(criterion, nn.BCEWithLogitsLoss):
        val_labels = np.concatenate(labels_list)
        val_pred = np.concatenate(pred_list)
        val_roc_auc = roc_auc_score(val_labels, val_pred)
        tb.add_scalar('{}/roc_auc'.format(tag), val_roc_auc, iteration)
        tb.add_pr_curve('{}/pr'.format(tag), val_labels, val_pred, iteration)

    return val_loss

def save_model(net: nn.Module, optimizer: optim.Optimizer,
               train_loss: float, val_loss: float,
               iteration: int, batch_size: int, epoch: int,
               path: str, train_videos_used: List, val_videos_used: List, test_videos_used: List):
    path = str(path)
    state = dict(net=net.state_dict(),
                 opt=optimizer.state_dict(),
                 train_loss=train_loss,
                 val_loss=val_loss,
                 iteration=iteration,
                 batch_size=batch_size,
                 epoch=epoch, 
                 train_videos_used=train_videos_used, 
                 val_videos_used=val_videos_used, 
                 test_videos_used=test_videos_used)
    torch.save(state, path)


# modified
def load_df(dfdc_df_path: str, ffpp_df_path: str, dfdc_faces_dir: str, ffpp_faces_dir: str, dataset: str) -> (pd.DataFrame, str):
    if dataset.startswith('dfdc'):
        df = pd.read_pickle(dfdc_df_path)
        root = dfdc_faces_dir
    elif dataset.startswith('ffpp'):
        df = pd.read_pickle(ffpp_df_path)
        folder = np.arange(len(df))
        df['folder']=folder
        root = ffpp_faces_dir
    else:
        raise NotImplementedError('Unknown dataset: {}'.format(dataset))
    return df, root

#modified
# TODO 
def get_split_df(df: pd.DataFrame, dataset: str, split: str) -> pd.DataFrame:
    if dataset == 'dfdc-35-5-10':
        st0 = np.random.get_state()
        np.random.seed(41)

        fake_videos_df = df[df['class']=='manipulated_sequences']
        videos = fake_videos_df.video.unique()
        rdm_videos_indices = np.random.choice(len(videos), len(videos), replace=False)
        train_videos = videos[rdm_videos_indices[:int(0.75*len(videos))]]
        valid_videos = videos[rdm_videos_indices[int(0.75*len(videos)):int(0.8*len(videos))]]
        test_videos = videos[rdm_videos_indices[int(0.8*len(videos)):]]

        real_videos_df = df[df['class']=='original_sequences']
        videos = real_videos_df.video.unique()    
        rdm_videos_indices = np.random.choice(len(videos), len(videos), replace=False)
        train_videos = np.append(train_videos, videos[rdm_videos_indices[:int(0.75*len(videos))]])
        valid_videos = np.append(valid_videos, videos[rdm_videos_indices[int(0.75*len(videos)):int(0.8*len(videos))]])
        test_videos = np.append(test_videos, videos[rdm_videos_indices[int(0.8*len(videos)):]])
        
        if split == 'train':
            videos_used = train_videos
            split_df = df[df.video.isin(videos_used)]
        elif split == 'val':
            videos_used = valid_videos
            split_df = df[df.video.isin(videos_used)]
        elif split == 'test':
            videos_used = test_videos
            split_df = df[df.video.isin(videos_used)]
        else:
            raise NotImplementedError('Unknown split: {}'.format(split))
    elif dataset.startswith('ffpp'):
        # Save random state
        st0 = np.random.get_state()
        # Set seed for this selection only
        np.random.seed(41)
        # Split on original videos
        crf = dataset.split('-')[1]

        random_youtube_videos = np.random.permutation(
            df[(df['source'] == 'youtube') & (df['quality'] == crf)]['video'].unique())
        print(len(random_youtube_videos))
        train_orig = random_youtube_videos[:720]
        val_orig = random_youtube_videos[720:720 + 140]
        test_orig = random_youtube_videos[720 + 140:]
        if split == 'train':
            split_df = pd.concat((df[df['original'].isin(train_orig)], df[df['video'].isin(train_orig)]), axis=0)
            print(len(split_df.video.unique()))
        elif split == 'val':
            split_df = pd.concat((df[df['original'].isin(val_orig)], df[df['video'].isin(val_orig)]), axis=0)
        elif split == 'test':
            split_df = pd.concat((df[df['original'].isin(test_orig)], df[df['video'].isin(test_orig)]), axis=0)
        else:
            raise NotImplementedError('Unknown split: {}'.format(split))

        if dataset.endswith('fpv'):
            fpv = int(dataset.rsplit('-', 1)[1][:-3])
            idxs = []
            for video in split_df['video'].unique():
                idxs.append(np.random.choice(split_df[split_df['video'] == video].index, fpv, replace=False))
            idxs = np.concatenate(idxs)
            split_df = split_df.loc[idxs]
        # Restore random state
        np.random.set_state(st0)
    else:
        raise NotImplementedError('Unknown dataset: {}'.format(dataset))
    return split_df, _#videos_used


def make_splits(dfdc_df: str, ffpp_df: str, dfdc_dir: str, ffpp_dir: str, dbs: Dict[str, List[str]]) -> Dict[str, Dict[str, Tuple[pd.DataFrame, str]]]:
    """
    Make split and return Dataframe and root
    :param
    dfdc_df: str, path to the DataFrame containing info on the faces extracted from the DFDC dataset with extract_faces.py
    ffpp_df: str, path to the DataFrame containing info on the faces extracted from the FF++ dataset with extract_faces.py
    dfdc_dir: str, path to the directory containing the faces extracted from the DFDC dataset with extract_faces.py
    ffpp_dir: str, path to the directory containing the faces extracted from the FF++ dataset with extract_faces.py
    dbs: {split_name:[split_dataset1,split_dataset2,...]}
                Example:
                {'train':['dfdc-35-5-15',],'val':['dfdc-35-5-15',]}
    :return: split_dict: dictonary containing {split_name: ['train', 'val'], splitdb: List(pandas.DataFrame, str)}
                Example:
                {'train, 'dfdc-35-5-15': (dfdc_train_df, 'path/to/dir/of/DFDC/faces')}
    """
    split_dict = {}
    full_dfs = {}

    for split_name, split_dbs in dbs.items():
        split_dict[split_name] = dict()
        for split_db in split_dbs:
            if split_db not in full_dfs:
                full_dfs[split_db] = load_df(dfdc_df, ffpp_df, dfdc_dir, ffpp_dir, split_db)
            full_df, root = full_dfs[split_db]
            split_df, folder = get_split_df(df=full_df, dataset=split_db, split=split_name)
            split_dict[split_name][split_db] = (split_df, root, folder)

    return split_dict



In [23]:
_ = get_split_df(df=pd.read_pickle("output/ffpp/dfs/from_video_0_to_video_0.pkl"), dataset="ffpp-c23-720-140-140", split='train')

1000
4320


In [7]:
net_class = getattr(fornet, "EfficientNetB4")
train_datasets = ["dfdc-35-5-10"]
val_datasets = ["dfdc-35-5-10"]
test_datasets = ["dfdc-35-5-10"]
#train_datasets = ["ffpp-c23-720-140-140"]
#val_datasets = ["ffpp-c23-720-140-140"]
dfdc_df_path = "output/dfdc/dfs/from_video_0_to_video_0.pkl"
ffpp_df_path = "output/ffpp/dfs/from_video_0_to_video_0.pkl"
dfdc_faces_dir = "output/dfdc/faces"
ffpp_faces_dir = "output/ffpp/faces"
face_policy = "scale"
face_size = 224

batch_size = 32
initial_lr = 1e-5
validation_interval = 500
patience = 10
max_num_iterations = 20000
initial_model = None
train_from_scratch = True

max_train_samples = -1
max_val_samples = 6000

log_interval = 100
num_workers = 6
device = torch.device('cuda:{:d}'.format(0)) if torch.cuda.is_available() else torch.device('cpu')
seed = 41

debug = False
suffix = "ENB4"

enable_attention = True

weights_folder = "weights/binclass/"
logs_folder = 'runs/binclass/'

In [8]:
# Random initialization
np.random.seed(seed)
torch.random.manual_seed(seed)

# Load net
net: nn.Module = net_class().to(device)

Loaded pretrained weights for efficientnet-b4


In [9]:
# Loss and optimizers
criterion = nn.BCEWithLogitsLoss()

min_lr = initial_lr * 1e-5
optimizer = optim.Adam(net.get_trainable_parameters(), lr=initial_lr)

#allows dynamic learning rate reducing based on some validation measurements.
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer,
    mode='min',
    factor=0.1,
    patience=patience,
    cooldown=2 * patience,
    min_lr=min_lr,
) 

tag = utils.make_train_tag(net_class=net_class,
                           traindb=train_datasets,
                           face_policy=face_policy,
                           patch_size=face_size,
                           seed=seed,
                           suffix=suffix,
                           debug=debug,
                           )

# Model checkpoint paths
bestval_path = os.path.join(weights_folder, tag, 'bestval.pth')
last_path = os.path.join(weights_folder, tag, 'last.pth')
periodic_path = os.path.join(weights_folder, tag, 'it{:06d}.pth')

os.makedirs(os.path.join(weights_folder, tag), exist_ok=True)

Parameters
{'face': 'scale',
 'net': 'EfficientNetB4',
 'seed': 41,
 'size': 224,
 'traindb': 'dfdc-35-5-10'}
Tag: net-EfficientNetB4_traindb-dfdc-35-5-10_face-scale_size-224_seed-41_ENB4


In [10]:
# Load model
val_loss = min_val_loss = 10
epoch = iteration = 0
net_state = None
opt_state = None
if initial_model is not None:
    # If given load initial model
    print('Loading model form: {}'.format(initial_model))
    state = torch.load(initial_model, map_location='cpu')
    net_state = state['net']
elif not train_from_scratch and os.path.exists(last_path):
    print('Loading model form: {}'.format(last_path))
    state = torch.load(last_path, map_location='cpu')
    net_state = state['net']
    opt_state = state['opt']
    iteration = state['iteration'] + 1
    epoch = state['epoch']
if not train_from_scratch and os.path.exists(bestval_path):
    state = torch.load(bestval_path, map_location='cpu')
    min_val_loss = state['val_loss']
if net_state is not None:
    incomp_keys = net.load_state_dict(net_state, strict=False)
    print(incomp_keys)
if opt_state is not None:
    for param_group in opt_state['param_groups']:
        param_group['lr'] = initial_lr
    optimizer.load_state_dict(opt_state)

In [11]:
# Initialize Tensorboard
logdir = os.path.join(logs_folder, tag)
if iteration == 0:
    # If training from scratch or initialization remove history if exists
    shutil.rmtree(logdir, ignore_errors=True)


In [12]:
# TensorboardX instance
tb = SummaryWriter(logdir=logdir)
if iteration == 0:
    dummy = torch.randn((1, 3, face_size, face_size), device=device)
    dummy = dummy.to(device)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        tb.add_graph(net, [dummy, ], verbose=False)

transformer = utils.get_transformer(face_policy=face_policy, patch_size=face_size,
                                    net_normalizer=net.get_normalizer(), train=True)
transformer

Compose([
  PadIfNeeded(always_apply=True, p=1.0, min_height=224, min_width=224, border_mode=0, value=0, mask_value=None),
  Resize(always_apply=True, p=1, height=224, width=224, interpolation=1),
  Downscale(always_apply=False, p=0.5, scale_min=0.5, scale_max=0.5, interpolation=0),
  Compose([
    HorizontalFlip(always_apply=False, p=0.5),
    OneOf([
      RandomBrightnessContrast(always_apply=False, p=0.5, brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), brightness_by_max=True),
      HueSaturationValue(always_apply=False, p=0.5, hue_shift_limit=(-10, 10), sat_shift_limit=(-30, 30), val_shift_limit=(-20, 20)),
    ], p=0.5),
    OneOf([
      ISONoise(always_apply=False, p=0.5, intensity=(0.1, 0.5), color_shift=(0.01, 0.05)),
      IAAAdditiveGaussianNoise(always_apply=False, p=0.5, loc=0, scale=(2.5500000000000003, 7.6499999999999995), per_channel=False),
    ], p=0.5),
    Downscale(always_apply=False, p=0.5, scale_min=0.7, scale_max=0.9, interpolation=1),
    ImageCompre

In [13]:
print('Loading data')
# Check if paths for DFDC and FF++ extracted faces and DataFrames are provided
for dataset in train_datasets:
    if dataset.split('-')[0] == 'dfdc' and (dfdc_df_path is None or dfdc_faces_dir is None):
        raise RuntimeError('Specify DataFrame and directory for DFDC faces for training!')
    elif dataset.split('-')[0] == 'ff' and (ffpp_df_path is None or ffpp_faces_dir is None):
        raise RuntimeError('Specify DataFrame and directory for FF++ faces for training!')
for dataset in val_datasets:
    if dataset.split('-')[0] == 'dfdc' and (dfdc_df_path is None or dfdc_faces_dir is None):
        raise RuntimeError('Specify DataFrame and directory for DFDC faces for validation!')
    elif dataset.split('-')[0] == 'ff' and (ffpp_df_path is None or ffpp_faces_dir is None):
        raise RuntimeError('Specify DataFrame and directory for FF++ faces for validation!')
for dataset in test_datasets:
    if dataset.split('-')[0] == 'dfdc' and (dfdc_df_path is None or dfdc_faces_dir is None):
        raise RuntimeError('Specify DataFrame and directory for DFDC faces for test!')
    elif dataset.split('-')[0] == 'ff' and (ffpp_df_path is None or ffpp_faces_dir is None):
        raise RuntimeError('Specify DataFrame and directory for FF++ faces for test!')


Loading data


In [14]:
# Load splits with the make_splits function
splits = make_splits(dfdc_df=dfdc_df_path, ffpp_df=ffpp_df_path,
                     dfdc_dir=dfdc_faces_dir, ffpp_dir=ffpp_faces_dir,
                     dbs={'train': train_datasets, 'val': val_datasets, 'test': test_datasets})

In [19]:
display(splits['train']['dfdc-35-5-10'][0])
display(splits['train']['dfdc-35-5-10'][1])
display(splits['test']['dfdc-35-5-10'][0])

,video,label,videosubject,class,source,quality,original,kp1x,kp1y,kp2x,...,kp5x,kp5y,kp6x,kp6y,conf,left,top,right,bottom,nfaces
facepath,,,,,,,,,,,,,,,,,,,,,
aagfhgtpmv.mp4/fr000_subj0.jpg,aagfhgtpmv.mp4,True,0,manipulated_sequences,videos,?,vudstovrck.mp4,232,223,330,...,142,273,350,231,0.926482,82,57,430,454,1
aagfhgtpmv.mp4/fr009_subj0.jpg,aagfhgtpmv.mp4,True,0,manipulated_sequences,videos,?,vudstovrck.mp4,226,209,330,...,142,251,360,249,0.940253,84,60,427,452,1
aagfhgtpmv.mp4/fr019_subj0.jpg,aagfhgtpmv.mp4,True,0,manipulated_sequences,videos,?,vudstovrck.mp4,225,215,330,...,140,248,356,249,0.932136,80,55,431,457,1
aagfhgtpmv.mp4/fr028_subj0.jpg,aagfhgtpmv.mp4,True,0,manipulated_sequences,videos,?,vudstovrck.mp4,221,205,326,...,143,240,359,273,0.888277,85,61,426,451,1
aagfhgtpmv.mp4/fr038_subj0.jpg,aagfhgtpmv.mp4,True,0,manipulated_sequences,videos,?,vudstovrck.mp4,210,213,316,...,147,240,365,268,0.892939,89,65,424,447,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eukvucdetx.mp4/fr260_subj0.jpg,eukvucdetx.mp4,True,0,manipulated_sequences,videos,?,gjypopglvi.mp4,206,207,301,...,159,252,357,268,0.954848,107,85,406,427,1
eukvucdetx.mp4/fr270_subj0.jpg,eukvucdetx.mp4,True,0,manipulated_sequences,videos,?,gjypopglvi.mp4,206,208,300,...,160,251,356,268,0.937686,106,86,405,427,1
eukvucdetx.mp4/fr279_subj0.jpg,eukvucdetx.mp4,True,0,manipulated_sequences,videos,?,gjypopglvi.mp4,205,209,295,...,165,251,353,264,0.898305,106,86,405,427,1


'output/dfdc/faces'

,video,label,videosubject,class,source,quality,original,kp1x,kp1y,kp2x,...,kp5x,kp5y,kp6x,kp6y,conf,left,top,right,bottom,nfaces
facepath,,,,,,,,,,,,,,,,,,,,,
acifjvzvpm.mp4/fr000_subj0.jpg,acifjvzvpm.mp4,True,0,manipulated_sequences,videos,?,kbvibjhfzo.mp4,228,220,271,...,212,228,306,228,0.995570,181,149,330,319,1
acifjvzvpm.mp4/fr009_subj0.jpg,acifjvzvpm.mp4,True,0,manipulated_sequences,videos,?,kbvibjhfzo.mp4,231,218,272,...,214,226,303,232,0.997065,184,150,329,316,1
acifjvzvpm.mp4/fr019_subj0.jpg,acifjvzvpm.mp4,True,0,manipulated_sequences,videos,?,kbvibjhfzo.mp4,229,216,272,...,211,225,305,226,0.949493,181,145,330,315,1
acifjvzvpm.mp4/fr028_subj0.jpg,acifjvzvpm.mp4,True,0,manipulated_sequences,videos,?,kbvibjhfzo.mp4,227,211,267,...,215,224,304,223,0.945526,181,141,330,310,1
acifjvzvpm.mp4/fr038_subj0.jpg,acifjvzvpm.mp4,True,0,manipulated_sequences,videos,?,kbvibjhfzo.mp4,228,210,269,...,215,221,304,226,0.929481,182,141,331,310,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
etohcvnzbj.mp4/fr260_subj0.jpg,etohcvnzbj.mp4,True,0,manipulated_sequences,videos,?,bdnaqemxmr.mp4,180,232,279,...,155,253,366,240,0.946465,85,61,427,451,1
etohcvnzbj.mp4/fr270_subj0.jpg,etohcvnzbj.mp4,True,0,manipulated_sequences,videos,?,bdnaqemxmr.mp4,177,227,272,...,162,244,366,255,0.959976,83,59,429,453,1
etohcvnzbj.mp4/fr279_subj0.jpg,etohcvnzbj.mp4,True,0,manipulated_sequences,videos,?,bdnaqemxmr.mp4,177,227,270,...,165,248,363,253,0.928147,95,73,417,440,1


In [23]:
train_dfs = [splits['train'][db][0] for db in splits['train']]
train_roots = [splits['train'][db][1] for db in splits['train']]
train_videos_used = [splits['train'][db][2] for db in splits['train']]
val_dfs = [splits['val'][db][0] for db in splits['val']]
val_roots = [splits['val'][db][1] for db in splits['val']]
val_videos_used = [splits['val'][db][2] for db in splits['val']]
test_dfs = [splits['test'][db][0] for db in splits['test']]
test_roots = [splits['test'][db][1] for db in splits['test']]
test_videos_used = [splits['test'][db][2] for db in splits['test']]

train_dataset = FrameFaceIterableDataset(roots=train_roots,
                                         dfs=train_dfs,
                                         scale=face_policy,
                                         num_samples=max_train_samples,
                                         transformer=transformer,
                                         size=face_size,
                                         )

val_dataset = FrameFaceIterableDataset(roots=val_roots,
                                       dfs=val_dfs,
                                       scale=face_policy,
                                       num_samples=max_val_samples,
                                       transformer=transformer,
                                       size=face_size,
                                       )

train_loader = DataLoader(train_dataset, num_workers=num_workers, batch_size=batch_size, )

val_loader = DataLoader(val_dataset, num_workers=num_workers, batch_size=batch_size, )

In [34]:
train_loader

In [35]:
next(iter(train_loader))

[tensor([[[[ 0.5707,  0.5536,  0.5536,  ...,  2.1804,  2.1804,  2.1804],
           [ 0.5364,  0.5536,  0.5364,  ...,  2.1804,  2.1804,  2.1804],
           [ 0.5193,  0.5364,  0.5536,  ...,  2.1804,  2.1804,  2.1804],
           ...,
           [ 0.1426,  0.1254,  0.1083,  ...,  1.7180,  1.7180,  1.7180],
           [ 0.1597,  0.1597,  0.1426,  ...,  1.7180,  1.7180,  1.7352],
           [ 0.1254,  0.1426,  0.1597,  ...,  1.7180,  1.7180,  1.7180]],
 
          [[ 1.2906,  1.2731,  1.2731,  ...,  1.7458,  1.7983,  1.7808],
           [ 1.2731,  1.2731,  1.2731,  ...,  1.7458,  1.7808,  1.7808],
           [ 1.2556,  1.2731,  1.2731,  ...,  1.7633,  1.7633,  1.7808],
           ...,
           [ 0.9930,  0.9755,  0.9755,  ...,  1.3431,  1.3256,  1.3431],
           [ 0.9930,  0.9930,  0.9930,  ...,  1.3256,  1.3081,  1.3256],
           [ 0.9580,  0.9930,  0.9930,  ...,  1.3256,  1.2906,  1.2906]],
 
          [[ 1.1411,  1.1237,  1.0888,  ...,  1.6465,  1.6640,  1.6640],
           [ 

In [21]:
train_dataset.transformer

Compose([
  PadIfNeeded(always_apply=True, p=1.0, min_height=224, min_width=224, border_mode=0, value=0, mask_value=None),
  Resize(always_apply=True, p=1, height=224, width=224, interpolation=1),
  Downscale(always_apply=False, p=0.5, scale_min=0.5, scale_max=0.5, interpolation=0),
  Compose([
    HorizontalFlip(always_apply=False, p=0.5),
    OneOf([
      RandomBrightnessContrast(always_apply=False, p=0.5, brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), brightness_by_max=True),
      HueSaturationValue(always_apply=False, p=0.5, hue_shift_limit=(-10, 10), sat_shift_limit=(-30, 30), val_shift_limit=(-20, 20)),
    ], p=0.5),
    OneOf([
      ISONoise(always_apply=False, p=0.5, intensity=(0.1, 0.5), color_shift=(0.01, 0.05)),
      IAAAdditiveGaussianNoise(always_apply=False, p=0.5, loc=0, scale=(2.5500000000000003, 7.6499999999999995), per_channel=False),
    ], p=0.5),
    Downscale(always_apply=False, p=0.5, scale_min=0.7, scale_max=0.9, interpolation=1),
    ImageCompre

In [33]:
print('Training samples: {}'.format(len(train_dataset)))
print('Validation samples: {}'.format(len(val_dataset)))

if len(train_dataset) == 0:
    print('No training samples. Halt.')

if len(val_dataset) == 0:
    print('No validation samples. Halt.')


stop = False
while not stop:

    # Training
    optimizer.zero_grad()

    train_loss = train_num = 0
    train_pred_list = []
    train_labels_list = []
    for train_batch in tqdm(train_loader, desc='Epoch {:03d} - Train loss {:03d} - Val loss {:03d}'.format(epoch, train_loss, val_loss), leave=False,
                            total=len(train_loader) // train_loader.batch_size):
        
        net.train()
        print(train_batch)
        batch_data, batch_labels = train_batch

        train_batch_num = len(batch_labels)
        train_num += train_batch_num
        train_labels_list.append(batch_labels.numpy().flatten())

        train_batch_loss, train_batch_pred = batch_forward(net, device, criterion, batch_data, batch_labels)
        train_pred_list.append(train_batch_pred.flatten())
        print('WESH')
        if torch.isnan(train_batch_loss):
            raise ValueError('NaN loss')

        train_loss += train_batch_loss.item() * train_batch_num

        # Optimization
        train_batch_loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        print('BIEN OU QUOI ?')
        
        # Logging
        if iteration > 0 and (iteration % log_interval == 0):
            train_loss /= train_num
            tb.add_scalar('train/loss', train_loss, iteration)
            tb.add_scalar('lr', optimizer.param_groups[0]['lr'], iteration)
            tb.add_scalar('epoch', epoch, iteration)

            # Checkpoint
            save_model(net, optimizer, train_loss, val_loss, iteration, batch_size, epoch, last_path)
            train_loss = train_num = 0
            print('ONELA')

        # Validation
        print('TABITES DANS LE COIN OU QUOI ?')
        if iteration > 0 and (iteration % validation_interval == 0):

            # Model checkpoint
            save_model(net, optimizer, train_loss, val_loss, iteration, batch_size, epoch,
                       periodic_path.format(iteration))

            # Train cumulative stats
            train_labels = np.concatenate(train_labels_list)
            train_pred = np.concatenate(train_pred_list)
            train_labels_list = []
            train_pred_list = []
            
            print('train_labels:',train_labels, 'train_pred:', train_pred)

            train_roc_auc = roc_auc_score(train_labels, train_pred)
            tb.add_scalar('train/roc_auc', train_roc_auc, iteration)
            tb.add_pr_curve('train/pr', train_labels, train_pred, iteration)

            # Validation
            val_loss = validation_routine(net, device, val_loader, criterion, tb, iteration, 'val')
            tb.flush()

            # LR Scheduler
            lr_scheduler.step(val_loss)
            
            print('ÇA DCHI KOI ?')
            # Model checkpoint
            if val_loss < min_val_loss:
                min_val_loss = val_loss
                save_model(net, optimizer, train_loss, val_loss, iteration, batch_size, epoch, bestval_path)

            # Attention
            if enable_attention and hasattr(net, 'get_attention'):
                net.eval()
                # For each dataframe show the attention for a real,fake couple of frames
                for df, root, sample_idx, tag in [
                    (train_dfs[0], train_roots[0], train_dfs[0][train_dfs[0]['label'] == False].index[0],
                     'train/att/real'),
                    (train_dfs[0], train_roots[0], train_dfs[0][train_dfs[0]['label'] == True].index[0],
                     'train/att/fake'),
                ]:
                    record = df.loc[sample_idx]
                    tb_attention(tb, tag, iteration, net, device, face_size, face_policy,
                                 transformer, root, record)

            if optimizer.param_groups[0]['lr'] == min_lr:
                print('Reached minimum learning rate. Stopping.')
                stop = True
                break

        iteration += 1

        if iteration > max_num_iterations:
            print('Maximum number of iterations reached')
            stop = True
            break

        # End of iteration

    epoch += 1

# Needed to flush out last events
tb.close()

print('Completed')

Epoch 000 - Train loss 000 - Val loss 010:   0%|          | 0/15 [00:00<?, ?it/s]

Training samples: 15482
Validation samples: 1024
[tensor([[[[ 2.2318,  2.2318,  2.2318,  ...,  0.6049,  0.6049,  0.6049],
          [ 2.2318,  2.2318,  2.2318,  ...,  0.6049,  0.6049,  0.6049],
          [ 2.2318,  2.2318,  2.2318,  ...,  0.6049,  0.6049,  0.6049],
          ...,
          [ 1.7865,  1.7865,  1.7865,  ...,  0.1426,  0.1426,  0.1426],
          [ 1.7865,  1.7865,  1.7865,  ...,  0.1426,  0.1426,  0.1426],
          [ 1.7865,  1.7865,  1.7865,  ...,  0.1426,  0.1426,  0.1426]],

         [[ 1.8333,  1.8333,  1.8333,  ...,  1.3081,  1.3081,  1.3081],
          [ 1.8333,  1.8333,  1.8333,  ...,  1.3081,  1.3081,  1.3081],
          [ 1.8333,  1.8333,  1.8333,  ...,  1.3081,  1.3081,  1.3081],
          ...,
          [ 1.3957,  1.3957,  1.3957,  ...,  1.0105,  1.0105,  1.0105],
          [ 1.3957,  1.3957,  1.3957,  ...,  1.0105,  1.0105,  1.0105],
          [ 1.3957,  1.3957,  1.3957,  ...,  1.0105,  1.0105,  1.0105]],

         [[ 1.7163,  1.7163,  1.7163,  ...,  1.1934,

WESH


Epoch 000 - Train loss 000 - Val loss 010:   7%|▋         | 1/15 [00:48<11:22, 48.75s/it]

BIEN OU QUOI ?
TABITES DANS LE COIN OU QUOI ?
[tensor([[[[-0.7822, -0.4739, -0.8164,  ..., -1.7069, -1.8610, -1.4843],
          [-0.6623, -0.7137, -0.7993,  ..., -1.8610, -1.7925, -1.6384],
          [-0.8164, -0.8678, -0.7479,  ..., -1.8439, -1.8610, -1.6213],
          ...,
          [-1.2274, -1.3302, -1.2959,  ..., -1.6042, -1.5699, -1.5699],
          [-1.0390, -1.1075, -1.2617,  ..., -1.6555, -1.5699, -1.9124],
          [-1.1760, -1.2959, -1.2788,  ..., -1.7925, -1.6213, -1.9809]],

         [[-0.8102, -0.4951, -0.8452,  ..., -1.5455, -1.7381, -1.3529],
          [-0.6877, -0.7402, -0.8277,  ..., -1.7031, -1.6331, -1.4930],
          [-0.8452, -0.8978, -0.7752,  ..., -1.6331, -1.7031, -1.4580],
          ...,
          [-1.3179, -1.4230, -1.3880,  ..., -1.6155, -1.5455, -1.5455],
          [-1.1253, -1.1954, -1.3529,  ..., -1.6681, -1.5455, -1.8957],
          [-1.2654, -1.3880, -1.3704,  ..., -1.8081, -1.5980, -1.9832]],

         [[-0.8110, -0.4973, -0.8458,  ..., -1.1247, -1

...

WESH


Epoch 000 - Train loss 000 - Val loss 010:  33%|███▎      | 5/15 [02:48<05:45, 34.51s/it]

BIEN OU QUOI ?
TABITES DANS LE COIN OU QUOI ?
[tensor([[[[-1.7754, -1.7754, -1.7925,  ..., -1.3987, -1.3815, -1.3815],
          [-1.7754, -1.7754, -1.7925,  ..., -1.3987, -1.3815, -1.3815],
          [-1.7925, -1.7925, -1.8097,  ..., -1.3987, -1.3815, -1.3815],
          ...,
          [-0.1486, -0.1486, -0.1486,  ..., -0.7650, -0.7650, -0.7650],
          [-0.1314, -0.1314, -0.1486,  ..., -0.7650, -0.7822, -0.7822],
          [-0.1314, -0.1314, -0.1486,  ..., -0.7650, -0.7822, -0.7822]],

         [[-1.8957, -1.8957, -1.8782,  ..., -1.4580, -1.4755, -1.4755],
          [-1.8957, -1.8957, -1.8782,  ..., -1.4580, -1.4755, -1.4755],
          [-1.8957, -1.8957, -1.8782,  ..., -1.4580, -1.4755, -1.4755],
          ...,
          [-0.5651, -0.5651, -0.5651,  ..., -1.0203, -1.0728, -1.0728],
          [-0.5476, -0.5476, -0.5651,  ..., -1.0203, -1.0378, -1.0378],
          [-0.5476, -0.5476, -0.5651,  ..., -1.0203, -1.0378, -1.0378]],

         [[-1.5953, -1.5953, -1.6302,  ..., -1.3513, -1

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f8d74e1cee0>
Traceback (most recent call last):
  File "/Users/lpdbrx/opt/anaconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()
  File "/Users/lpdbrx/opt/anaconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/Users/lpdbrx/opt/anaconda3/lib/python3.8/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/Users/lpdbrx/opt/anaconda3/lib/python3.8/multiprocessing/popen_fork.py", line 44, in wait
    if not wait([self.sentinel], timeout):
  File "/Users/lpdbrx/opt/anaconda3/lib/python3.8/multiprocessing/connection.py", line 930, in wait
    ready = selector.select(timeout)
  File "/Users/lpdbrx/opt/anaconda3/lib/python3.8/selectors.py", line 415, in select
    fd_event_list = self._selector.poll(timeout)


KeyboardInterrupt: 